# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [8]:
from selenium import webdriver  # permite criar o navegador
from selenium.webdriver.common.keys import Keys  # permite escrever no navegador
from selenium.webdriver.common.by import By  # permite selecionar items no navegador
import pandas as pd

In [9]:
# opções do navegador
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # rodar em segundo plano

# abrir o navegador
navegador = webdriver.Chrome(options=options)

# Passo 1: Pegar a cotação do dólar
# entrar no Google
navegador.get('https://www.google.com/')

# pesquisar no Google por "cotação dólar"
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação do dólar')
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegar a contação que tá no Google
cotacao_dolar = float(navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value'))
print(f'Dólar vale: R${cotacao_dolar:.2f}')

# Passo 2: Pegar a cotação do euro
# entrar no Google
navegador.get('https://www.google.com/')

# pesquisar no Google por "cotação euro"
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação do euro')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegar a contação que tá no Google
cotacao_euro = float(navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value'))
print(f'Euro vale: R${cotacao_euro:.2f}')

# Passo 3: Pegar a cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')

# pegar a cotação que tá no site
cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = float(cotacao_ouro.replace(',', '.'))
print(f'Ouro vale: R${cotacao_ouro:.2f}')

navegador.quit()

Dólar vale: R$5.20
Euro vale: R$5.13
Ouro vale: R$287.16


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [10]:
# Passo 4: Atualizar a base de dados (atualizando o preço de compra e o de venda)
df = pd.read_excel('Produtos.xlsx')
# display(df)

- Atualizando os preços e o cálculo do Preço Final

In [11]:
# atualizar a coluna de cotação
df.loc[df['Moeda'] == 'Dólar', 'Cotação'] = cotacao_dolar
df.loc[df['Moeda'] == 'Euro', 'Cotação'] = cotacao_euro
df.loc[df['Moeda'] == 'Ouro', 'Cotação'] = cotacao_ouro

# atualizar a coluna de preço de compra
df.loc[df['Moeda'] == 'Dólar', 'Preço de Compra'] = df['Preço Original'] * df['Cotação']
df.loc[df['Moeda'] == 'Euro', 'Preço de Compra'] = df['Preço Original'] * df['Cotação']
df.loc[df['Moeda'] == 'Ouro', 'Preço de Compra'] = df['Preço Original'] * df['Cotação']

# atualizar a coluna de preço de venda
df['Preço de Venda'] = df['Preço de Compra'] * df['Margem']

### Formatando valores no dataframe

In [12]:
df['Cotação'] = round(df['Cotação'], 2)
df['Preço de Compra'] = round(df['Preço de Compra'], 2)
df['Preço de Venda'] = round(df['Preço de Venda'], 2)

In [13]:
# display(df)

### Agora vamos exportar a nova base de preços atualizada

In [14]:
# Passo 5: Exportar a base de preços atualizada
df.to_excel('Produtos Novo.xlsx', index=False)